In [434]:
import os, sys
import json
import pprint
import numpy as np

In [435]:
PATH_TO_COURSE_DATA = os.path.join(os.getcwd(), "data", "courses_stick")

PATH_TO_COURSES_TXT = os.path.join(PATH_TO_COURSE_DATA, "courses_txt")
PATH_TO_COURSE_JSON = os.path.join(os.getcwd(), "data", "courses.json")

In [436]:
courses_json = json.load(open(PATH_TO_COURSE_JSON, 'r'))

In [437]:
courses = { course["short_name"] : course for course in courses_json}

In [438]:
# courses

In [439]:
def get_txt_from_file(path : str):
#     print(path)
    with open(path, "r", encoding="utf-8") as file:
        text = file.read()
        return text
    
    return None

In [440]:
# txt = get_txt_from_file(os.path.join(PATH_TO_COURSES_TXT, "AlgGeom.txt"))

In [441]:
field_of_knowledge_list = os.listdir(PATH_TO_COURSE_DATA)
field_of_knowledge_list

['Алгебра']

In [442]:
for field_of_knowledge in field_of_knowledge_list:
    path_to_sub_dir = os.path.join(PATH_TO_COURSE_DATA, field_of_knowledge)
    for sub_dir in os.listdir(path_to_sub_dir):
        path_to_dir_file = os.path.join(path_to_sub_dir, sub_dir)
        for file_name in os.listdir(path_to_dir_file):
            short_name = file_name.split('.')[0]
            
            path_to_file = os.path.join(path_to_dir_file, file_name)
            text = get_txt_from_file(path_to_file)
            courses[short_name]['description'] = text

In [443]:
courses = {short_name : course for short_name, course in courses.items() \
           if course['description'][:4] != "http" and \
           course['description'] != 'Описание отсутсвует'}

In [444]:
print(len(courses))
# courses

61


In [445]:
list_courses = [(short_name, course) for short_name, course in courses.items()]
list_courses_txt = [course['description'] for short_name, course in courses.items()]

In [446]:
# list_courses
# list_courses_txt

In [447]:
#  LDA

In [448]:
from tqdm import tqdm_notebook as tqdm

# from sklearn.externals import joblib
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA
from sklearn.model_selection import train_test_split
import pickle

import nltk
from nltk.corpus import stopwords
# from pymystem3 import Mystem
import re

from os.path import join
from glob import glob

In [449]:
# stop_words
# nltk.download('stopwords')


# add other stop words!
stop_words = []

nltk_stop_words = stopwords.words('russian')
stop_words.extend(nltk_stop_words)
print(f"nltk_stop_words: len={len(nltk_stop_words)}")

# RussianStopWords.txt
path_now_2 = os.path.join(os.getcwd(), "data", "stop_words", "RussianStopWords.txt")
with open(path_now_2, "r") as f:
    stop_words_2 = [word.strip() for word in f.read().split("\n")]
#     print(stop_words_2)
    stop_words.extend(stop_words_2)
    print(f"RussianStopWords: len={len(stop_words_2)}")

# stopwords-ru.json
path_now_3 = os.path.join(os.getcwd(), "data", "stop_words", "stopwords-ru.json")
stop_words_3 = json.load(open(path_now_3, "r"))
stop_words.extend(stop_words_3)
print(f"stopwords-ru.json: len={len(stop_words_3)}")


print(f"all words: len={len(stop_words)}")


nltk_stop_words: len=151
RussianStopWords: len=421
stopwords-ru.json: len=559
all words: len=1131


In [450]:
# load text!
train_names = list_courses_txt
pass
# train_names

In [451]:
print('Got %d stopwords.\nGot %d texts in training corpus.'%(len(stop_words), len(train_names))) 

Got 1131 stopwords.
Got 61 texts in training corpus.


In [452]:
#'training' (tf-)idf vectorizer.
tf_idf = TfidfVectorizer()

X = tf_idf.fit_transform(train_names)
#getting idfs
idfs = tf_idf.idf_
#sorting out too rare and too common words
#original 1.3 and 7
# 2 6
lower_thresh = 1.3
upper_thresh = 7
not_often = idfs > lower_thresh
not_rare = idfs < upper_thresh

mask = not_often * not_rare

# good_words = np.array(tf_idf.get_feature_names())[mask]
good_words = np.array(tf_idf.get_feature_names_out())[mask]

#deleting punctuation as well.
cleaned_tf = []
for word in good_words:
    word = re.sub("^(\d+\w*$|_+)", "", word)
    
    if len(word) == 0:
        continue
    cleaned_tf.append(word)
print("Len of original vocabulary: %d\nAfter filtering: %d"%(idfs.shape[0], len(cleaned_tf)))

Len of original vocabulary: 3665
After filtering: 3537


In [453]:
# print(idfs)
# print(cleaned)
# print(X)

In [454]:
import re
def all_cyrillic(text):
    all_ru = True
    for letter in text:
        all_ru = all_ru and bool(re.search('[а-яА-Я]', letter))
    return all_ru

def delete_non_letters(words):
    new_words = []
    for word in words:
        new_word = "".join(c for c in word if c.isalpha())
        if new_word != '':
            new_words.append(new_word)
    return new_words


def delete_stopwords(words, stop_words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        
        if word not in stop_words:
            new_words.append(word)

    return new_words

def delete_no_ru(words):
    new_words = []
    for word in words:
        
        if all_cyrillic(word):
            new_words.append(word)

    return new_words

def small_word(words, min_size = 4):
    new_words = []
    for word in words:
        
        if len(word) >= min_size:
            new_words.append(word)

    return new_words
    

def normalize(words, stop_words):
    cleaned = words
    
    old_len = len(cleaned)
    cleaned = delete_non_letters(cleaned)
    print("Len of original vocabulary: %d\nAfter filtering: %d"%(old_len, len(cleaned)))
    
    old_len = len(cleaned)
    cleaned = delete_stopwords(cleaned, stop_words)
    print("Len of original vocabulary: %d\nAfter filtering: %d"%(old_len, len(cleaned)))
    
    old_len = len(cleaned)
    cleaned = delete_no_ru(cleaned)
    print("Len of original vocabulary: %d\nAfter filtering: %d"%(old_len, len(cleaned)))
    
    old_len = len(cleaned)
    cleaned = small_word(cleaned, 4)
    print("Len of original vocabulary: %d\nAfter filtering: %d"%(old_len, len(cleaned)))
    
    return cleaned
    
    

In [455]:
cleaned = normalize(cleaned_tf, stop_words)

Len of original vocabulary: 3537
After filtering: 3537
Len of original vocabulary: 3537
After filtering: 3343
Len of original vocabulary: 3343
After filtering: 2753
Len of original vocabulary: 2753
After filtering: 2706


In [461]:
# cleaned

In [315]:
from pymystem3 import Mystem
# !pip install pymystem3


In [422]:
#Stemming
m = Mystem()
stemmed = set()
voc_len = len(cleaned)
for i in tqdm(range(voc_len)):
    word = cleaned.pop()
    stemmed_word = m.lemmatize(word)[0]
    stemmed.add(stemmed_word)
    
stemmed = list(stemmed)
print('After stemming: %d'%(len(stemmed)))

/tmp/ipykernel_155324/3833386003.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(voc_len)):


  0%|          | 0/2706 [00:00<?, ?it/s]

After stemming: 1483


In [489]:
# stemmed

In [490]:
#training count vectorizer
voc = {word : i for i, word in enumerate(stemmed)}

count_vect = CountVectorizer(#list_courses_txt,
                            #stop_words=stop_words,
                            vocabulary=voc
                            )
X = count_vect.fit_transform(list_courses_txt)

dataset = count_vect.fit_transform(train_names)

In [492]:
# dataset

In [426]:
lda = LDA(n_components = 60, max_iter=100, n_jobs=6, learning_method='batch', verbose=1)
lda.fit(dataset) 

iteration: 1 of max_iter: 100
iteration: 2 of max_iter: 100
iteration: 3 of max_iter: 100
iteration: 4 of max_iter: 100
iteration: 5 of max_iter: 100
iteration: 6 of max_iter: 100
iteration: 7 of max_iter: 100
iteration: 8 of max_iter: 100
iteration: 9 of max_iter: 100
iteration: 10 of max_iter: 100
iteration: 11 of max_iter: 100
iteration: 12 of max_iter: 100
iteration: 13 of max_iter: 100
iteration: 14 of max_iter: 100
iteration: 15 of max_iter: 100
iteration: 16 of max_iter: 100
iteration: 17 of max_iter: 100
iteration: 18 of max_iter: 100
iteration: 19 of max_iter: 100
iteration: 20 of max_iter: 100
iteration: 21 of max_iter: 100
iteration: 22 of max_iter: 100
iteration: 23 of max_iter: 100
iteration: 24 of max_iter: 100
iteration: 25 of max_iter: 100
iteration: 26 of max_iter: 100
iteration: 27 of max_iter: 100
iteration: 28 of max_iter: 100
iteration: 29 of max_iter: 100
iteration: 30 of max_iter: 100
iteration: 31 of max_iter: 100
iteration: 32 of max_iter: 100
iteration: 33 of 

LatentDirichletAllocation(max_iter=100, n_components=60, n_jobs=6, verbose=1)

In [494]:
lda.components_

array([[0.01666667, 0.01666667, 0.01666667, ..., 0.01666667, 0.01666667,
        0.01666667],
       [0.01666667, 0.01666667, 0.01666667, ..., 0.01666667, 0.01666667,
        0.01666667],
       [0.01666667, 0.01666667, 0.01666667, ..., 0.01666667, 0.01666667,
        0.01666667],
       ...,
       [0.01666667, 0.01666667, 0.01666667, ..., 0.01666667, 1.01666667,
        0.01666667],
       [0.01666667, 0.01666667, 0.01666667, ..., 0.01666667, 0.01666667,
        0.01666667],
       [0.01666667, 0.01666667, 0.01666667, ..., 0.01666667, 0.01666667,
        0.01666667]])

In [472]:
# "ли" in voc

In [473]:
# TESTING

In [474]:
#Let\`s do simple interface class
class TopicModeler(object):
    '''
    Inteface object for CountVectorizer + LDA simple
    usage.
    '''
    def __init__(self, count_vect, lda):
        '''
        Args:
             count_vect - CountVectorizer object from sklearn.
             lda - LDA object from sklearn.
        '''
        self.lda = lda
        self.count_vect = count_vect
        self.count_vect.input = 'content'
        
    def __call__(self, text):
        '''
        Gives topics distribution for a given text
        Args:
             text - raw text via python string.
        returns: numpy array - topics distribution for a given text.
        '''
        vectorized = self.count_vect.transform([text])
        lda_topics = self.lda.transform(vectorized)
        return lda_topics
    
    def get_keywords(self, text, n_topics=3, n_keywords=5):
        '''
        For a given text gives n top keywords for each of m top texts topics.
        Args:
             text - raw text via python string.
             n_topics - int how many top topics to use.
             n_keywords - how many top words of each topic to return.
        returns:
                list - of m*n keywords for a given text.
        '''
        lda_topics = self(text)
        lda_topics = np.squeeze(lda_topics, axis=0)
        n_topics_indices = lda_topics.argsort()[-n_topics:][::-1]
        
        top_topics_words_dists = []
        for i in n_topics_indices:
            top_topics_words_dists.append(self.lda.components_[i])
        
        keywords = np.zeros(shape=(n_keywords*n_topics, self.lda.components_.shape[1]))
        for i,topic in enumerate(top_topics_words_dists):
            n_keywords_indices = topic.argsort()[-n_keywords:][::-1]
            for k,j in enumerate(n_keywords_indices):
                keywords[i * n_keywords + k, j] = 1
        keywords = self.count_vect.inverse_transform(keywords)
        keywords = [keyword[0] for keyword in keywords]
        return keywords  

In [475]:
tm = TopicModeler(count_vect, lda) 

In [476]:
text = list_courses_txt[2]
# print(text)

In [478]:
key_words = tm.get_keywords(text, n_topics=1, n_keywords=20)
print(key_words) 
print(text)

ValueError: X has 1483 features, but LatentDirichletAllocation is expecting 3665 features as input.

In [499]:
# gen sim
# !pip install gensim

In [515]:
# dataset
m_ = Mystem()

def lemmatize(word):
    global m_
    #print(m_.lemmatize(word))
    return m_.lemmatize(word)[0]

In [540]:
docs = []
for doc in tqdm(train_names):
    list_words = []
    for word in doc.split(" "):
        if word == '' or word == '\n':
            continue
        list_words.append(lemmatize(word))
    docs.append(' '.join(list_words))
    
# docs

/tmp/ipykernel_155324/679783651.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for doc in tqdm(train_names):


  0%|          | 0/61 [00:00<?, ?it/s]

In [521]:
cv = CountVectorizer(vocabulary=voc)

cv_docs = cv.fit_transform(docs)


In [541]:
cv.vocabulary

{'цикл': 0,
 'теор': 1,
 'сноп': 2,
 'вплотную': 3,
 'ровно': 4,
 'градуировать': 5,
 'снабжать': 6,
 'косой': 7,
 'математик': 8,
 'ман': 9,
 'фиксированный': 10,
 'порождаемость': 11,
 'дважды': 12,
 'систеиы': 13,
 'превосходность': 14,
 'подъем': 15,
 'неизмеримый': 16,
 'очередь': 17,
 'циклический': 18,
 'джан': 19,
 'классифицироваться': 20,
 'хофф': 21,
 'жорданова': 22,
 'иметь': 23,
 'анонс': 24,
 'минимальный': 25,
 'несимметричность': 26,
 'мотивный': 27,
 'относительно': 28,
 'компонент': 29,
 'заключительный': 30,
 'кручение': 31,
 'рений': 32,
 'двумерный': 33,
 'обозначаться': 34,
 'становиться': 35,
 'пред': 36,
 'векторный': 37,
 'тейхмюллер': 38,
 'квазитреугольный': 39,
 'плоскость': 40,
 'называть': 41,
 'слева': 42,
 'интеграл': 43,
 'дополнение': 44,
 'якобианов': 45,
 'посвящать': 46,
 'доступно': 47,
 'определять': 48,
 'произведение': 49,
 'дерево': 50,
 'обертывать': 51,
 'коразмерность': 52,
 'использоваться': 53,
 'характеря': 54,
 'полупервичный': 55,
 'ве

array([[0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [542]:
from gensim import models

def words_in_doc(doc):
    doc_ = []
    for idx, word_count in enumerate(doc):
        if word_count:
            doc_.append((idx, word_count))
    return doc_

corpus = [words_in_doc(doc) for doc in cv_docs.toarray()]
id2word = {idx: word for idx, word in enumerate(cv.vocabulary)}



In [543]:
corpus

[[(1, 1),
  (12, 1),
  (46, 1),
  (57, 1),
  (62, 1),
  (115, 1),
  (122, 1),
  (123, 1),
  (129, 1),
  (140, 1),
  (148, 3),
  (170, 1),
  (172, 1),
  (173, 1),
  (174, 1),
  (175, 1),
  (177, 1),
  (190, 4),
  (199, 1),
  (211, 1),
  (272, 1),
  (328, 1),
  (329, 4),
  (336, 1),
  (353, 1),
  (375, 1),
  (380, 1),
  (394, 1),
  (504, 1),
  (515, 1),
  (518, 7),
  (547, 1),
  (563, 1),
  (578, 1),
  (583, 1),
  (625, 2),
  (626, 1),
  (636, 6),
  (637, 1),
  (651, 1),
  (663, 4),
  (674, 1),
  (679, 1),
  (687, 2),
  (714, 2),
  (749, 3),
  (754, 8),
  (761, 1),
  (783, 2),
  (819, 1),
  (825, 1),
  (865, 16),
  (867, 5),
  (892, 1),
  (916, 1),
  (928, 1),
  (936, 1),
  (954, 1),
  (958, 1),
  (971, 1),
  (1006, 1),
  (1014, 2),
  (1018, 1),
  (1021, 1),
  (1022, 1),
  (1037, 1),
  (1054, 3),
  (1073, 1),
  (1074, 2),
  (1083, 1),
  (1089, 1),
  (1121, 2),
  (1129, 1),
  (1175, 2),
  (1176, 3),
  (1196, 2),
  (1198, 1),
  (1222, 1),
  (1248, 1),
  (1255, 2),
  (1261, 1),
  (1262, 1),

In [544]:
lda_model = models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=15)

In [545]:
sorted(lda_model.print_topics(), key = lambda topic: float(topic[1].split('*')[0])) #.split(' + ')[0]

[(5,
  '0.016*"алгебра" + 0.013*"теория" + 0.013*"категория" + 0.012*"функтор" + 0.011*"форма" + 0.011*"кольцо" + 0.011*"группа" + 0.010*"поле" + 0.010*"курс" + 0.008*"проективный"'),
 (12,
  '0.019*"группа" + 0.011*"схема" + 0.009*"теория" + 0.009*"групповой" + 0.008*"редуктивный" + 0.008*"алгебраический" + 0.008*"курс" + 0.007*"многообразие" + 0.006*"простой" + 0.006*"геометрия"'),
 (10,
  '0.022*"форма" + 0.022*"кольцо" + 0.020*"алгебраический" + 0.019*"поле" + 0.015*"идеал" + 0.014*"теорема" + 0.013*"являться" + 0.012*"курс" + 0.011*"схема" + 0.011*"теория"'),
 (3,
  '0.024*"кольцо" + 0.015*"алгебраический" + 0.014*"поле" + 0.014*"форма" + 0.013*"теория" + 0.012*"число" + 0.010*"теорема" + 0.010*"курс" + 0.009*"модуль" + 0.009*"алгебра"'),
 (14,
  '0.024*"группа" + 0.018*"кольцо" + 0.018*"теорема" + 0.017*"теория" + 0.016*"многообразие" + 0.014*"категория" + 0.013*"представление" + 0.013*"проективный" + 0.013*"курс" + 0.012*"дивизор"'),
 (6,
  '0.027*"поле" + 0.026*"теория" + 0.019

In [546]:
lda_model.get_document_topics(cv_docs.toarray()[0])

IndexError: invalid index to scalar variable.